# Linear Regression

In this notebook, we will learn how to apply Linear regression for predicting the heating load requirements (Y1) of buildings as a function of building parameters (Xs).

The attached dataset is taken from the [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Energy+efficiency).

To run this code, you will need the following python packages:
* numpy
* pandas
* openpyxl
* scikit-learn

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install openpyxl

In [ ]:
# First, we load the dataset using pandas
df = pd.read_excel("Energy_Efficiency.xlsx", engine = 'openpyxl')
# Remove any unnamed columns (might occur due to difference in pandas readers)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
# Remove any row with NaNs
df = df.dropna(how='all')
# Drop Y2 (as we only consider Y1 for regression)
df = df.drop('Y2', axis=1)

In [ ]:
# next, we will split the dataframe into a training and testing splits with a 70% / 30% ratio
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.3, random_state=42) # Random is fixed for reproducability

In [ ]:
# Now lets display a few rows from the training data
df_train

In [ ]:
# Then lets view some statistics
df_train.describe()

In [ ]:
# Now we will extract the models input and targets from both the training and testing dataframes
def extract_Xy(df):
    df_numpy = df.to_numpy()
    return df_numpy[:, :-1], df_numpy[:, -1]

X_train, y_train = extract_Xy(df_train)
X_test, y_test = extract_Xy(df_test)

## Linear Regression via Scikit-Learn

In [ ]:
# Then we test the linear regression using Scikit-learn's implementation
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(X_train, y_train)

In [ ]:
# Using scikit-learn's MSE function, we can compute the training and testing error for our model
from sklearn.metrics import mean_squared_error

y_train_predict = model.predict(X_train)
training_error = mean_squared_error(y_train, y_train_predict)
print(f"Training Error: {training_error} (RMS: {training_error**0.5})")
y_test_predict = model.predict(X_test)
testing_error = mean_squared_error(y_test, y_test_predict)
print(f"Testing Error: {testing_error} (RMS: {testing_error**0.5})")

#Note: We also display the Root Mean Square error (RMS) since it is more intuitive to compare with the dataset statistics (diplayed using df_train.describe())


In [ ]:
%%timeit
LinearRegression().fit(X_train, y_train)
# Here we are measuring the training time to compare with our implementation below

## Linear Regression from Scratch

In [ ]:
def our_mean_square_error(true, predicted):
    #TODO: implement this function to match Scikit-learn's mean_square_error
    #Note: both true & predicted will be float numpy arrays
    pass

In [ ]:
print(f"{our_mean_square_error( np.array([  1, 0]), np.array([1,   0]) ) = }") # Should be 0
print(f"{our_mean_square_error( np.array([  0, 1]), np.array([1,   0]) ) = }") # Should be 1
print(f"{our_mean_square_error( np.array([0.5, 0]), np.array([1, 0.5]) ) = }") # Should be 0.25

In [ ]:
class OurLinearRegression:
    def _prepare_inputs(self, X):
        # Here, we add a new input with value 1 to each example. It will be multipled by the bias
        ones = np.ones((X.shape[0], 1), dtype=X.dtype)
        return np.concatenate((ones, X), axis=1)

    def fit(self, X, y):
        X = self._prepare_inputs(X) # First, we prepare the inputs
        #TODO: compute and store the model weights into self.w
        # Note: you can use numpy function and do not use "numpy.linalg.lstsq" or "numpy.linalg.pinv"
        # To compute a square matrix's inverse, you can use "numpy.linalg.inv".
        # A more stable option to compute "numpy.linalg.inv(A) @ b" is using "numpy.linalg.solve(A, b)" 
        
        # Return self to match the behavior of Scikit-Learn's LinearRegression fit()
        return self
    
    def predict(self, X):
        X = self._prepare_inputs(X) # First, we prepare the inputs
        #TODO: Compute and return the predictions given X

In [ ]:
# Now, you can train your model
our_model = OurLinearRegression().fit(X_train, y_train)

In [ ]:
# Using your MSE function, you can compute the training and testing error for our model
y_train_predict = our_model.predict(X_train)
training_error = our_mean_square_error(y_train, y_train_predict)
print(f"Training Error: {training_error} (RMS: {training_error**0.5})")
y_test_predict = our_model.predict(X_test)
testing_error = our_mean_square_error(y_test, y_test_predict)
print(f"Testing Error: {testing_error} (RMS: {testing_error**0.5})")

In [ ]:
%%timeit
OurLinearRegression().fit(X_train, y_train)
# Now, you can compare the time of our implementation with Scikit-Learn's. What is your conclusion?

In [ ]:
#TODO: Write your conclusion about your implementation's performance and training time